In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
735,tt1564367,80000000,214918407,Just Go With It,Jennifer Aniston|Adam Sandler|Nicole Kidman|Ba...,Dennis Dugan,Sometimes a guy's best wingman... is a wingwoman,"In Just Go With It, a plastic surgeon, romanci...",117,Romance|Comedy,Columbia Pictures|Happy Madison Productions,2/10/2011,6.3,2011
600,tt0910970,180000000,521311860,WALLÂ·E,Ben Burtt|Elissa Knight|Jeff Garlin|Fred Willa...,Andrew Stanton,An adventure beyond the ordinar-E.,WALLÂ·E is the last robot left on an Earth tha...,98,Animation|Family,Walt Disney Pictures|Pixar Animation Studios,6/22/2008,7.6,2008
198,tt2980648,22000000,88880821,The Hundred-Foot Journey,Helen Mirren|Manish Dayal|Om Puri|Charlotte Le...,Lasse HallstrÃ¶m,Life's greatest journey begins with the first ...,A story centered around an Indian family who m...,122,Drama,Amblin Entertainment|Harpo Films|Participant M...,8/6/2014,7.2,2014


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

# преобразуем столбцы, в которых есть несколько значений, в списки
to_list = lambda s: s.split("|")
data["cast"] = data["cast"].apply(to_list)
data["director"] = data["director"].apply(to_list)
data["genres"] = data["genres"].apply(to_list)
data["production_companies"] = data["production_companies"].apply(to_list)

# преобразуем столбец release_date в формат datetime
data["release_date"] = pd.to_datetime(data["release_date"])

# добавим признак, показывающий месяц, в котором вышел фильм
months = "Январь Февраль Март Апрель Май Июнь Июль Август Сентябрь Октябрь Ноябрь Декабрь".split()
data["release_month"] = data["release_date"].apply(lambda x: months[x.month - 1])

# добавим признак, показывающий прибыль
profit = data["revenue"] - data["budget"]
data.insert(loc=3, column="profit", value=profit)

# добавим также признак, содержащий название фильма и его id
formatted_title = data["original_title"] + " (" + data["imdb_id"] + ")"
data.insert(loc=5, column="formatted_title", value=formatted_title)

def get_first_movie_title(df):
    """Возвращает отформатированное название первого фильма в датафрейме"""
    return df.iloc[0]["formatted_title"]

def get_most_common_values(s):
    """
    Возвращает отсортированный список самых встречающихся значений 
    в объекте Series, каждый элемент которого содержит список значений
    """
    c = Counter()
    for values in s.values:
        for value in values:
            c[value] += 1
    return c.most_common()

def explode_groupby(df, column_name):
    """
    Создает строку для каждого элемента списка в столбце и 
    группирует датафрейм по значению в этом столбце
    """
    return df.explode(column_name).groupby(column_name)

def get_first_item_by_value(s):
    """Возвращает название элемента с наибольшим значением в Series"""
    return s.sort_values(ascending=False).index[0]

data.head(3)

,imdb_id,budget,revenue,profit,original_title,formatted_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,release_month
0,tt0369610,150000000,1513528810,1363528810,Jurassic World,Jurassic World (tt0369610),"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]","[Universal Studios, Amblin Entertainment, Lege...",2015-06-09,6.5,2015,Июнь
1,tt1392190,150000000,378436354,228436354,Mad Max: Fury Road,Mad Max: Fury Road (tt1392190),"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...",[George Miller],What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,"[Action, Adventure, Science Fiction, Thriller]","[Village Roadshow Pictures, Kennedy Miller Pro...",2015-05-13,7.1,2015,Май
2,tt2908446,110000000,295238201,185238201,Insurgent,Insurgent (tt2908446),"[Shailene Woodley, Theo James, Kate Winslet, A...",[Robert Schwentke],One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,"[Adventure, Science Fiction, Thriller]","[Summit Entertainment, Mandeville Films, Red W...",2015-03-18,6.3,2015,Март


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
answers["1"] = "Pirates of the Caribbean: On Stranger Tides (tt1298650)"

In [7]:
df = data[data["budget"] == data["budget"].max()]
get_first_movie_title(df)

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
answers["2"] = "Gods and Generals (tt0279111)"

In [9]:
df = data[data["runtime"] == data["runtime"].max()]
get_first_movie_title(df)

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [10]:
answers["3"] = "Winnie the Pooh (tt1449283)"

In [11]:
df = data[data["runtime"] == data["runtime"].min()]
get_first_movie_title(df)

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [12]:
answers["4"] = 110

In [13]:
round(data["runtime"].mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [14]:
answers["5"] = 107

In [15]:
round(data["runtime"].median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [16]:
answers["6"] = "Avatar (tt0499549)"

In [17]:
df = data[data["profit"] == data["profit"].max()]
get_first_movie_title(df)

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [18]:
answers["7"] = "The Lone Ranger (tt1210819)"

In [19]:
df = data[data["profit"] == data["profit"].min()]
get_first_movie_title(df)

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [20]:
answers["8"] = 1478

In [21]:
len(data[data["profit"] > 0])

1478

ВАРИАНТ 2

In [22]:
data[data["profit"] > 0].shape[0]

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [23]:
answers["9"] = "The Dark Knight (tt0468569)"

In [24]:
m_2008 = data[data["release_year"] == 2008]
df = m_2008[m_2008["revenue"] == m_2008["revenue"].max()]
get_first_movie_title(df)

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [25]:
answers["10"] = "The Lone Ranger (tt1210819)"

In [26]:
m_2012_14 = data[data["release_year"].isin([2012, 2013, 2014])]
df = m_2012_14[m_2012_14["profit"] == m_2012_14["profit"].min()]
get_first_movie_title(df)

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [27]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

In [28]:
answers["11"] = "Drama"

In [29]:
get_most_common_values(data["genres"])[0][0]

'Drama'

ВАРИАНТ 2

In [30]:
genres = [genre for genres in data["genres"].values for genre in genres]
unique_genres = set(genres)
genres_count = {genre: genres.count(genre) for genre in unique_genres}
most_common_genres = sorted(genres_count, key=genres_count.get, reverse=True)
most_common_genres[0]

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [31]:
answers["12"] = "Drama"

In [32]:
profit_movies = data[data["profit"] > 0]
get_most_common_values(profit_movies["genres"])[0][0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [33]:
answers["13"] = "Peter Jackson"

In [34]:
directors_by_revenue = explode_groupby(data, "director")["revenue"].sum()
get_first_item_by_value(directors_by_revenue)

'Peter Jackson'

ВАРИАНТ 2

In [35]:
directors_by_revenue = data.explode("director").pivot_table(index="director", values="revenue", aggfunc="sum")
directors_by_revenue.sort_values("revenue", ascending=False).index[0]

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [36]:
answers["14"] = "Robert Rodriguez"

In [37]:
s = explode_groupby(data, "director")["genres"].sum().apply(lambda x: x.count("Action"))
get_first_item_by_value(s)

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [38]:
answers["15"] = "Chris Hemsworth"

In [39]:
m_2012 = data[data["release_year"] == 2012]
s = explode_groupby(m_2012, "cast")["revenue"].sum()
get_first_item_by_value(s)

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [40]:
answers["16"] = "Matt Damon"

In [41]:
m_high_budget = data[data["budget"] > data["budget"].mean()]
s = explode_groupby(m_high_budget, "cast")["budget"].count()
get_first_item_by_value(s)

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [42]:
answers["17"] = "Action"

In [43]:
actors_by_genre = explode_groupby(data, "cast")["genres"].sum()
cage_genres = actors_by_genre["Nicolas Cage"]
c = Counter()
for genre in cage_genres:
    c[genre] += 1
c.most_common()[0][0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [44]:
answers["18"] = "K-19: The Widowmaker (tt0267626)"

In [45]:
df_by_companies = data.explode("production_companies")
m_paramount = df_by_companies[df_by_companies["production_companies"] == "Paramount Pictures"]
df = m_paramount.sort_values("profit")
get_first_movie_title(df)

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [46]:
answers["19"] = 2015

In [47]:
s = data.groupby("release_year")["revenue"].sum()
get_first_item_by_value(s)

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [48]:
answers["20"] = 2014

In [49]:
exploded_by_companies = data.explode("production_companies")
m_warner_bros = exploded_by_companies[exploded_by_companies["production_companies"].str.contains("Warner Bros")]
s = m_warner_bros.groupby("release_year")["profit"].sum()
get_first_item_by_value(s)

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [50]:
answers["21"] = "Сентябрь"

In [51]:
get_first_item_by_value(data["release_month"].value_counts())

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [52]:
answers["22"] = 450

In [53]:
len(data[data["release_month"].isin(["Июнь", "Июль", "Август"])])

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [54]:
answers["23"] = "Peter Jackson"

In [55]:
m_winter = data[data["release_month"].isin(["Декабрь", "Январь", "Февраль"])]
s = explode_groupby(m_winter, "director")["release_month"].count()
get_first_item_by_value(s)

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [56]:
answers["24"] = "Four By Two Productions"

In [57]:
group = explode_groupby(data, "production_companies")["original_title"]
s = group.apply(lambda x: np.mean(x.str.len()))
get_first_item_by_value(s)

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [58]:
answers["25"] = "Midnight Picture Show"

In [59]:
group = explode_groupby(data, "production_companies")["overview"]
s = group.apply(lambda x: np.mean(x.str.split().str.len()))
get_first_item_by_value(s)

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [60]:
answers["26"] = "Inside Out, The Dark Knight, 12 Years a Slave"

In [61]:
best_one_percent = data[data["vote_average"] > data["vote_average"].quantile(0.99)]
best_one_percent["original_title"]

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [62]:
answers["27"] = "Daniel Radcliffe & Rupert Grint"

In [63]:
cast_combinations_series = data["cast"].apply(lambda x: list(itertools.combinations(x, 2)))
cast_pairs = [pair for pairs in cast_combinations_series.values for pair in pairs]
c = Counter()
for pair in cast_pairs:
    if (pair[1], pair[0]) in c:
        c[(pair[1], pair[0])] += 1
    else:
        c[pair] += 1
pair = c.most_common()[0][0]
f"{pair[0]} & {pair[1]}"

'Daniel Radcliffe & Rupert Grint'

# Submission

In [64]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [65]:
# и убедиться что ни чего не пропустил)
len(answers)

27